<a href="https://colab.research.google.com/github/Hyeok-Jun-Yoon/AI_Plus/blob/main/%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C_5%EC%A3%BC%EC%B0%A8_AI_2%EA%B8%B0_%EC%9C%A4%ED%98%81%EC%A4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본과제: 외부 블로그의 정보와 함께 챗봇 구현하기

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 13.7 MB/s eta 0:00:0

In [ ]:
import bs4 #BeautifulSoup 라이브러리, HTML,XML파일을 파싱하고 탐색
import requests
from langchain import hub #LangChain 내에서 사용할 수 있는 다양한 컴포넌트관리
from langchain_chroma import Chroma #텍스트 임베딩을 저장 , 임베딩을 기반으로 유사한 문서를 검색,인덱싱
from langchain_openai import ChatOpenAI #openAI사용하여 대화형 응답을 생성하는 기능을 제공
from langchain_openai import OpenAIEmbeddings #텍스트를 벡터임베딩으로 변환,유사도검색 및 분석가능
from langchain_community.document_loaders import WebBaseLoader #지정된 URL에서 웹 페이지 콘텐츠를 추출,데이터로드,가공
from langchain_text_splitters import RecursiveCharacterTextSplitter #긴텍스트를 작은 단위로 나눔
from langchain.schema import Document #문서를 나타내는 클래스
from google.colab import userdata
from langchain import PromptTemplate

## [MYCODE] 데이터 준비

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    #CSS클래스-HTML요소파싱
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("editedContent")
        )
    ),
)
loader.encoding = "utf-8" # 한글인코딩추가
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로

## [MYCODE] 수상작 작품 내용만 추출

In [ ]:
# 'chunk_size'는 각 덩어리의 최대 크기(문자 수)를 정의하고,
# 'chunk_overlap'은 각 덩어리 간의 겹치는 문자의 수를 설정
'''
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=500
)

splits = text_splitter.split_documents(docs) #문서가 여러 덩어리로 나뉘게 됨
'''

# docs 리스트에서 각 Document 객체의 page_content 속성을 \xa0로 분리
result = [doc.page_content.split('\xa0') for doc in docs]
# 결과 출력
for item in result:
    #print(item[1:6])  # 각 문서의 내용이 \xa0로 분리된 리스트 형태로 출력됨
    premier = ''.join(item[1:6])

#print(premier)
for doc in docs:
  doc.page_content = premier  # 새로운 page_content로 업데이트

In [ ]:
print(docs)

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도 눈에 띄게 빨라질 수 있어요. 언어 공부의 복잡함을 단순하게 만들어주는 Lexi Note가 요긴한 학습 도우미가 되었기 때문입니다.사용한 기술 스택 -FE(프론트엔드): React, Tesseract.js, React-Quill, HTML, CSS, JavaScript-BE(백엔드): Java, Spring Boot, MariaDB🎖️우수상[우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스서비스 제작자: 인트(배정연, 한지수)💡C는 이번 학기 처음으로 자취를 시작했습니다. 원래부터 작은 벌레에 민감했던 C. 자취방에서 자주 나

## [MYCODE] 벡터화 및 Retriever 작업 수행

In [ ]:
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(api_key=userdata.get('api_key'))
)
# vectorstore 객체를 retriever로 변환하여 검색할 수 있는 형태로 설정
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
print(retriever)

tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a33ae4be6b0> search_kwargs={'k': 6}


In [ ]:
find_msg = "수상작 대상,우수상,입선"
retrieved_docs = retriever.invoke(find_msg) # retriever를 사용하여 사용자의 메시지(user_msg)와 관련된 문서들을 검색
print(retrieved_docs)

# retriever 한 결과값이 중복된 내용이 있어 중복 제거
unique_docs = []
seen = set()

for doc in retrieved_docs:
    if doc.page_content not in seen:
        unique_docs.append(doc)
        seen.add(doc.page_content)

print(unique_docs)


[Document(id='f5195ca4-19b5-4961-afe3-540cc034592d', metadata={'category': 'category_8', 'id': 'entry_8'}, page_content='공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터'), Document(id='99ee6e3a-46f4-404d-9997-82af9b6a45c1', metadata={'category': 'category_7', 'id': 'entry_7'}, page_content='입선[BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼서비스 제작자: 블로티(이은주, 한명수, 황준영)💡F는 T대학에 교환학생으로 온 외국인입니다. K팝 문화가 좋아 한국에 왔지만, 말도 잘 통하지 않고 문화 차이도 있어서 한국 캠퍼스 생활이 쉽지만은 않은데요. 한국어로 수업을 듣고 한국 친구도 사귀고 싶지만 한국 대학의 학사 일정과 과제 등 여러 시스템이 달라서 적응하기 어려웠습니다. 한국어가 서툴다보니 수업에서 제공되는 다양한 정보를 이해하기 어려웠고, 중요한 공지사항을 놓치기도 했습니다. 한국인 친구를 먼저 사귀는 것이 급선무라고 생각했죠.<BLOTIE>는 외국인 학생이 가진 이런 문제를 해결하고, 보다 친밀하고 자유로운 글로벌 캠퍼스 문화를 조성하기 위해 만들어진 플랫폼입니다. ‘1:1 매칭 기능’으로 교내 외국인 학생과 한국인 학생을 쉽게 연결할 수 있어요. 서로의 문화와 언어를 자연스럽게 배우고 교류할 수 있습니다. 관심사, 언어 능력, 성격 등을 기반으로 이뤄지기 때문에 일회성 매칭에 대한 걱정도 줄어들어요. 매칭된 학생들은 플랫폼 내에서 ‘실시간 채팅과 피드 기능’으로 자유롭게 소통할 수 있고, 다양한 주제로 만남을 이어갈 수 있습니다.외국인은 한국어 실력을 향상할 수 있고 한국인은 외국어를 실생활에서 연습할 기회를 얻을 수 있어요. BLOTIE를 통해 캠퍼스 내에 보다 다양한 문화가 공존하고, 학생들 간의 유대감이 강화되

## [MYCODE] prompt 생성

In [ ]:
# Define the summarization prompt template
summarization_template = PromptTemplate(
    input_variables=["user_msg,text"],
    template="{user_msg}: {text}"
)
print(summarization_template)

input_variables=['text', 'user_msg'] input_types={} partial_variables={} template='{user_msg}: {text}'


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) # 각 문서의 내용을 "\n\n"로 구분하여 하나의 문자열로 합침

In [ ]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘"
summary = summarization_template.format(user_msg=user_msg,text=format_docs(unique_docs))
print(summary)

ALL-in 코딩 공모전 수상작들을 요약해줘: 공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터

입선[BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼서비스 제작자: 블로티(이은주, 한명수, 황준영)💡F는 T대학에 교환학생으로 온 외국인입니다. K팝 문화가 좋아 한국에 왔지만, 말도 잘 통하지 않고 문화 차이도 있어서 한국 캠퍼스 생활이 쉽지만은 않은데요. 한국어로 수업을 듣고 한국 친구도 사귀고 싶지만 한국 대학의 학사 일정과 과제 등 여러 시스템이 달라서 적응하기 어려웠습니다. 한국어가 서툴다보니 수업에서 제공되는 다양한 정보를 이해하기 어려웠고, 중요한 공지사항을 놓치기도 했습니다. 한국인 친구를 먼저 사귀는 것이 급선무라고 생각했죠.<BLOTIE>는 외국인 학생이 가진 이런 문제를 해결하고, 보다 친밀하고 자유로운 글로벌 캠퍼스 문화를 조성하기 위해 만들어진 플랫폼입니다. ‘1:1 매칭 기능’으로 교내 외국인 학생과 한국인 학생을 쉽게 연결할 수 있어요. 서로의 문화와 언어를 자연스럽게 배우고 교류할 수 있습니다. 관심사, 언어 능력, 성격 등을 기반으로 이뤄지기 때문에 일회성 매칭에 대한 걱정도 줄어들어요. 매칭된 학생들은 플랫폼 내에서 ‘실시간 채팅과 피드 기능’으로 자유롭게 소통할 수 있고, 다양한 주제로 만남을 이어갈 수 있습니다.외국인은 한국어 실력을 향상할 수 있고 한국인은 외국어를 실생활에서 연습할 기회를 얻을 수 있어요. BLOTIE를 통해 캠퍼스 내에 보다 다양한 문화가 공존하고, 학생들 간의 유대감이 강화되고, 서로 다른 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통

## [MYCODE] 최종 결과

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=userdata.get('api_key'))
response = llm.invoke(user_msg+summary)
print(response.content)

## ALL-in 코딩 공모전 수상작 요약

### 대상
**[Lexi Note]**  
- **서비스 설명**: 언어 학습을 효과적으로 지원하는 필기 웹 서비스. 단어를 드래그하면 네이버 사전으로 연결되고, 필요한 경우 파파고 번역기를 통해 긴 문장을 쉽게 이해할 수 있음. 할일 목록과 스케줄 템플릿 제공.
- **기술 스택**: React, Tesseract.js, Java, Spring Boot 등.

### 우수상
**[우리집 히어로즈]**  
- **서비스 설명**: 자취생들이 겪는 벌레 문제를 해결하기 위한 매칭 서비스. 사용자와 벌레 퇴치 히어로를 안전하게 매칭하여 실시간 알림을 제공하며, 사용자 리뷰 기능으로 서비스 품질 향상.
- **기술 스택**: React Native, Spring Boot 등.

**[에코 클래스룸]**  
- **서비스 설명**: 교수와 학생 모두에게 수업 중 의견과 질문을 익명으로 전달할 수 있는 플랫폼. 실시간 피드백과 퀴즈 기능을 통해 학생의 이해도를 체크하고 수업 속도를 조절할 수 있음.
- **기술 스택**: Flutter, Spring Boot 등.

### 입선작
**[BLOTIE]**  
- **서비스 설명**: 외국인과 한국인 학생 간의 문화 교류를 지원하는 매칭 플랫폼. 관심사와 언어 능력 기반으로 1:1 매칭을 통해 교류하며, 챗 기능 제공.
- **기술 스택**: Spring Boot, React Native 등.

**[Crewing]**  
- **서비스 설명**: 신입생이 연합 동아리를 쉽게 찾을 수 있도록 도와주는 플랫폼. 생년, 성별, 관심 분야 입력 시 맞춤 동아리를 추천하고, 리크루팅 과정을 한 번에 처리 가능.
- **기술 스택**: Spring Boot, Redis 등.

**[학교생활 매니저]**  
- **서비스 설명**: 학교 생활을 효율적으로 관리하는 앱. 일정 관리, 과제 및 시험 알림, 성적 예측 및 기록 기능을 제공하여 체계적인 관리 도와줌.
- **기술 스택**: Fl